In [1]:
# This is the training script for fine-tuning bert on 
# unaltered GPT data and manually labelled data
# For better performance/generalization, look for augmented dataset
# Read README.md for comments and details.

In [2]:
# all classes

classes = ["banking","valuation","household","real estate","corporate","external","sovereign","technology", "climate", "energy", "health", "eu"]


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,top_k_accuracy_score
import math
import pickle
from datasets import Dataset

In [4]:
# load bert-based and finbert
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(classes))
finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert', use_fast =True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
# weights transfer for encoder layers only 
finbert_weights = finbert.state_dict()
model_weights = model.state_dict()
del finbert_weights["bert.pooler.dense.weight"]
del finbert_weights["bert.pooler.dense.bias"]
del finbert_weights["classifier.weight"]
del finbert_weights["classifier.bias"]
finbert_weights["bert.pooler.dense.weight"] = model_weights["bert.pooler.dense.weight"]
finbert_weights["bert.pooler.dense.bias"] = model_weights["bert.pooler.dense.bias"]
finbert_weights["classifier.weight"] = model_weights["classifier.weight"]
finbert_weights["classifier.bias"] = model_weights["classifier.bias"]

model.load_state_dict(finbert_weights)


<All keys matched successfully>

In [6]:
# flatten to one list for all 3

# manual labelled
with open('train_data.pickle', 'rb') as file:
    train = pickle.load(file)

# gpt labelled p1
with open('gpt.pickle', 'rb') as file:
    gpt = pickle.load(file)

# gpt labelled p2
with open('gpt_p2.pickle', 'rb') as file:
    gpt2 = pickle.load(file)
    
gpt = [item for sublist in gpt for item in sublist]
gpt2 = [item for sublist in gpt2 for item in sublist]

mixed = gpt + gpt2

In [7]:
print(len(mixed))

2458


In [8]:
# change to sampling methods instead of argmax if treat GPT-3 data as probabilities.
sample = 1

text_max = [item["text"] for i in range(sample) for item in train]
label_max = [np.argmax(item["dist"]) for i in range(sample) for item in train]

text_max_mixed = [item["text"] for i in range(sample) for item in mixed]
label_max_mixed = [np.argmax(item["dist"]) for i in range(sample) for item in mixed]

comb_text = text_max_mixed
comb_label = label_max_mixed

In [9]:
assert len(comb_text) == len(comb_label)

In [10]:
# random shuffle
import random
temp = list(zip(comb_text, comb_label))
random.shuffle(temp)
comb_text, comb_label = zip(*temp)

In [11]:
print(len(comb_text))

2458


In [12]:
train_text, test_text = comb_text[:math.ceil(len(comb_text)*0.9)], comb_text[math.ceil(len(comb_text)*0.9):]
train_label, test_label = comb_label[:math.ceil(len(comb_label)*0.9)], comb_label[math.ceil(len(comb_label)*0.9):]

In [13]:
train_dataset = Dataset.from_dict({"text":train_text, "label":train_label})
test_dataset = Dataset.from_dict({"text":test_text, "label":test_label})

In [14]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

100%|██████████| 1/1 [00:00<00:00, 90.89ba/s]


In [15]:
train_dataset["input_ids"]

tensor([[  101,  1037,  2488,  ...,     0,     0,     0],
        [  101,  9944,  2181,  ...,     0,     0,     0],
        [  101,  2045,  2323,  ...,     0,     0,     0],
        ...,
        [  101,  4826,  1010,  ...,     0,     0,     0],
        [  101,  2714, 10069,  ...,     0,     0,     0],
        [  101,  4613,  2011,  ...,     0,     0,     0]])

In [16]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    top3 = top_k_accuracy_score(labels, pred.predictions,k=3)
    top2 = top_k_accuracy_score(labels, pred.predictions,k=2)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'top3': top3,
         'top2': top2
    }

training_args = TrainingArguments(
    
    output_dir='./results',
    learning_rate=2e-5,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    metric_for_best_model="accuracy",
    evaluation_strategy='epoch',
    save_strategy = "epoch",
    logging_dir='./logs',
    save_total_limit = 1, # Only last 5 models are saved. Older ones are deleted.
    load_best_model_at_end=True,
)
    
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [17]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 2213
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2780
  5%|▍         | 138/2780 [00:22<06:56,  6.34it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

{'eval_loss': 2.2275032997131348, 'eval_accuracy': 0.2653061224489796, 'eval_f1': 0.05849293465392536, 'eval_precision': 0.06293809700372548, 'eval_recall': 0.08994708994708994, 'eval_top3': 0.5959183673469388, 'eval_top2': 0.46938775510204084, 'eval_runtime': 0.3134, 'eval_samples_per_second': 781.818, 'eval_steps_per_second': 12.764, 'epoch': 1.0}


Model weights saved in ./results\checkpoint-139\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-2618] due to args.save_total_limit
 10%|▉         | 277/2780 [00:46<06:13,  6.70it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 10%|█         | 278/2780 [00:46<06:13,  6.70it/s]Saving model checkpoint to ./results\checkpoint-278
Configuration saved in ./results\checkpoint-278\config.json


{'eval_loss': 1.8715399503707886, 'eval_accuracy': 0.4163265306122449, 'eval_f1': 0.12697716513505986, 'eval_precision': 0.10321704578594341, 'eval_recall': 0.16820987654320987, 'eval_top3': 0.6408163265306123, 'eval_top2': 0.5306122448979592, 'eval_runtime': 0.2999, 'eval_samples_per_second': 816.823, 'eval_steps_per_second': 13.336, 'epoch': 2.0}


Model weights saved in ./results\checkpoint-278\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-3080] due to args.save_total_limit
 15%|█▍        | 416/2780 [01:08<05:49,  6.76it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 15%|█▌        | 417/2780 [01:08<05:49,  6.76it/s]Saving model checkpoint to ./results\checkpoint-417
Configuration saved in ./results\checkpoint-417\config.json


...

Model weights saved in ./results\checkpoint-2641\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-2502] due to args.save_total_limit
100%|█████████▉| 2779/2780 [07:25<00:00,  7.01it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 64

100%|██████████| 2780/2780 [07:26<00:00,  7.01it/s]Saving model checkpoint to ./results\checkpoint-2780
Configuration saved in ./results\checkpoint-2780\config.json


{'eval_loss': 2.555901288986206, 'eval_accuracy': 0.5551020408163265, 'eval_f1': 0.3685013382171906, 'eval_precision': 0.37002416972382646, 'eval_recall': 0.37137419529892646, 'eval_top3': 0.7795918367346939, 'eval_top2': 0.6979591836734694, 'eval_runtime': 0.3009, 'eval_samples_per_second': 814.113, 'eval_steps_per_second': 13.292, 'epoch': 20.0}


Model weights saved in ./results\checkpoint-2780\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-2641] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-1529 (score: 0.5795918367346938).
100%|██████████| 2780/2780 [07:28<00:00,  6.20it/s]

{'train_runtime': 448.3402, 'train_samples_per_second': 98.72, 'train_steps_per_second': 6.201, 'train_loss': 0.49576307299325795, 'epoch': 20.0}


TrainOutput(global_step=2780, training_loss=0.49576307299325795, metrics={'train_runtime': 448.3402, 'train_samples_per_second': 98.72, 'train_steps_per_second': 6.201, 'train_loss': 0.49576307299325795, 'epoch': 20.0})

In [18]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 64
 75%|███████▌  | 3/4 [00:00<00:00, 19.79it/s]C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 4/4 [00:00<00:00, 17.92it/s]


{'eval_loss': 2.1347901821136475,
 'eval_accuracy': 0.5795918367346938,
 'eval_f1': 0.370981538194653,
 'eval_precision': 0.3877801120448179,
 'eval_recall': 0.3759495308554448,
 'eval_top3': 0.7836734693877551,
 'eval_top2': 0.726530612244898,
 'eval_runtime': 0.2973,
 'eval_samples_per_second': 824.003,
 'eval_steps_per_second': 13.453,
 'epoch': 20.0}